In [1]:
import pandas as pd
import random
import os
import copy
import numpy as np
import json
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import seaborn as sns
from functools import reduce
from constants import diffMappingToScore, questions
#from utils import read_json

In [2]:
#####################
# This notebook computes the following measures
#
#  - Comprehension accuracy
#  - Comprehension efficiency (response time)
#  - Perceived difficulty
#
#######################

In [3]:
def addQuestionInfo(allData,questions):
    
    #change the type of questionID to integer
    allData['questionID'] = allData['questionID'].astype('int')
    
    #extend the columns of questionnaireData with those in DataFrame(questions) based the common question ID
    allData = allData.merge(pd.DataFrame(questions), left_on=['questionID'], right_on=['id'])
    
    return allData

In [25]:
def read_json(path):
    with open(path, encoding='utf-8') as inFile:
        return json.load(inFile)
    

def answeringAccuracy(allData,grouper):
    
    #compute answer accuracy as 1 if x['questionAnswer']==x['ExpectedAnswer'] else 0
    allData["AnswerAccuracy"] = allData.apply(lambda x: 1 if x['questionAnswer']==x['ExpectedAnswer'] else 0, axis=1)
    
    #group by grouper and compute mean AnswerAccuracy
    allData = allData.groupby(grouper,as_index=False).agg({'AnswerAccuracy':'mean'})
    
    return allData


def timeInterval(x):
    return x.iloc[-1]-x.iloc[0]

def responseTime(allData):
    
    #goup by questionID and Typ1, Type2 and Type3
    #Grouping by questionID allows us to compute timeInterval() by substracting the last timestamp for the question from the first one
    #Grouping by 'Type1','Type2','Type3' is done just to keep track of these attributes for further grouping. The grouping by questionID is already at the most fine-grained question level
    allData = allData.groupby(['questionID','participant','Type1','Type2','Type3'],as_index=False).agg(timeInterval=('questionTimestamp', timeInterval))

    return allData


def diffToScore(x):
    return diffMappingToScore[x]

def perceivedDifficulty(allData,grouper):
    
    allData['difficultyScore'] = allData.apply(lambda x: diffToScore(x['questionAnswer']),axis=1)
    
    #group by grouper
    allData = allData.groupby(grouper,as_index=False).agg({'difficultyScore':'mean'})

    return allData


In [26]:
#######
#
# Loading and Preprocessing 
#
######

In [27]:
#loading the data
directory = "data/"
pattern = re.compile("EyeMind_EyeMind_(.+?)_collected-data.json")
allData = None


for subdir, dirs, files in os.walk(directory):
    for file in files:  
        if pattern.match(file):
            filename = os.path.join(subdir, file)
            data = pd.DataFrame(read_json(filename)["processedGazeData"]["gazeData"])
           
            #data["participant"] = re.search('data/(.+?)\\\EyeMind_EyeMind_',filename).group(1) #for windows
            data["participant"] = re.search('data/(.+?)/EyeMind_EyeMind_',filename).group(1) # for mac
            allData = pd.concat([allData,data],axis=0)
            
allData = allData.reset_index(drop=True)

In [30]:
#previewing the data
allData.head()

,questionTimestamp,questionEventType,questionText,questionAnswer,questionPosition,questionID,eventSource,Timestamp,validLeft,validRight,...,leftY,rightX,rightY,x,y,leftDistance,rightDistance,tabName,element,participant
0,1.673612e+12,questionOnset,"The activities ""Register arrival of container...",,0.0,1,questionnaire,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,SP10-no
1,NaN,None,None,None,NaN,None,eye-tracker,956173.450,1.0,1.0,...,399.0,782.0,425.0,766.0,412.0,621.869141,616.152832,F1_Offload_Container.bpmn,Process_1jt1th5,SP10-no
2,NaN,None,None,None,NaN,None,eye-tracker,956181.814,1.0,1.0,...,270.0,765.0,313.0,747.5,291.5,621.258545,616.167297,F1_Offload_Container.bpmn,Process_1jt1th5,SP10-no
3,NaN,None,None,None,NaN,None,eye-tracker,956190.119,1.0,1.0,...,204.0,740.0,214.0,732.5,209.0,621.120178,616.018127,F1_Offload_Container.bpmn,,SP10-no
4,NaN,None,None,None,NaN,None,eye-tracker,956198.445,1.0,1.0,...,193.0,743.0,220.0,734.0,206.5,621.527405,615.918335,F1_Offload_Container.bpmn,,SP10-no


In [31]:
allData["participant"].unique()

array(['SP10-no', 'SP2-no', 'KP7-no', 'SP14-no', 'SP20-no', 'SP6-no',
       'KP3-no', 'SP15-no', 'SP21-no', 'SP7-no', 'KP2-no', 'SP11-no',
       'SP3-no', 'KP6-no', 'KP24-no', 'SP8-no', 'KP10-no', 'KP9-no',
       'KP14-no', 'KP20-no', 'KP8-no', 'KP15-no', 'KP21-no', 'SP9-no',
       'KP11-no', 'KP16-no', 'KP22-no', 'SP18-no', 'KP12-no', 'SP19-no',
       'KP13-no', 'KP17-no', 'KP23-no', 'SP16-no', 'SP22-no', 'SP4-no',
       'KP1-no', 'SP12-no', 'KP5-no', 'KP18-no', 'SP13-no', 'SP1-no',
       'KP4-no', 'KP19-no', 'SP17-no', 'SP5-no'], dtype=object)

In [32]:
#######################
#
# Comprehension Accuracy
#
#######################

In [33]:
#measure specific pre-processing

In [34]:
# select rows with questionEventType='questionOffset' and 'questionID' is in the questions considered for the analysis
ansAccData = allData.loc[(allData['questionEventType']=='questionOffset') & (allData['questionID'].isin([str(question["id"]) for question in questions]))].copy(deep=True)

#add question info
ansAccData = addQuestionInfo(ansAccData,questions)

#Select control-flow question type
ansAccData = ansAccData.loc[(ansAccData['Type2'] == 'Control-flow')] 

In [35]:
"""
2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
"""
ansAccData = ansAccData.drop(ansAccData[(allData['participant'] == 'SP11-no') & (ansAccData['Type3'] == 'Exclusiveness')].index)
#ansAcc_new.shape

<ipython-input-35-9e443a4d9eea>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ansAccData = ansAccData.drop(ansAccData[(allData['participant'] == 'SP11-no') & (ansAccData['Type3'] == 'Exclusiveness')].index)


In [36]:
##Check for nan values
#print(ansAcc[ansAcc.isna().any(axis=1)])

In [37]:
# Example query: For each participant, we obtain one aggregated score for local questions and one aggregate score for global questions
answeringAccuracy(ansAccData,['participant','Type1'])

,participant,Type1,AnswerAccuracy
0,KP1-no,Global,0.75
1,KP1-no,Local,1.00
2,KP10-no,Global,1.00
3,KP10-no,Local,1.00
4,KP11-no,Global,0.25
...,...,...,...
87,SP7-no,Local,1.00
88,SP8-no,Global,0.75
89,SP8-no,Local,1.00
90,SP9-no,Global,0.25


In [38]:
# Descriptives

In [39]:
answeringAccuracy(ansAccData,['Type1'])

,Type1,AnswerAccuracy
0,Global,0.739130
1,Local,0.978261


In [40]:
#V2
#Calculate mean per participant
ansAcc_participant = answeringAccuracy(ansAccData,['participant','Type1','Type2'])
#Calculate descriptives
ansAcc_participant.groupby('Type1').agg({'AnswerAccuracy':'mean'})

,AnswerAccuracy
Type1,
Global,0.739130
Local,0.978261


In [41]:
# Inferentials

In [42]:
# Get a mean value for each participant
ansAcc_participant = answeringAccuracy(ansAccData,['participant','Type1','Type2'])

In [43]:
#control-flow
ansAcc_cflow_Global = ansAcc_participant.loc[(ansAcc_participant["Type1"]=='Global')]
ansAcc_cflow_Local = ansAcc_participant.loc[(ansAcc_participant["Type1"]=='Local')]
ansAccuracy_cflow_merge = ansAcc_cflow_Global.merge(ansAcc_cflow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')

print(len(ansAcc_cflow_Global), len(ansAcc_cflow_Local)) 
print(len(ansAccuracy_cflow_merge))

stats.wilcoxon(ansAccuracy_cflow_merge['AnswerAccuracy_global'], ansAccuracy_cflow_merge['AnswerAccuracy_local'],alternative='less')

46 46
46


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=0.0, pvalue=6.108386720026067e-07)

In [44]:
######################
#
# Comprehension efficiency (Response time)
#
######################

In [45]:
#measure specific pre-processing

In [46]:
# select rows with 'questionID' is in the questions considered for the analysis
resTimeData = allData.loc[(allData['questionID'].isin([str(question["id"]) for question in questions]))].copy(deep=True)
    
#add question info
resTimeData = addQuestionInfo(resTimeData,questions)

#Select control-flow question type
resTimeData = resTimeData.loc[(resTimeData['Type2'] == 'Control-flow')] 

In [47]:
"""
2 rows are removed for participant SP11. This concerns the questions local and global (control-flow) Exclusiveness, since the participant 
skipped the answer for the local Exclusiveness question by mistake. Hence the duration and accuracy were biased!
"""
resTimeData = resTimeData.drop(resTimeData[(resTimeData['participant'] == 'SP11-no') & (resTimeData['Type3'] == 'Exclusiveness')].index)
print(resTimeData.shape)

(732, 38)


In [48]:
#Example of query: time interval taken to answer each question by each participant
responseTime(resTimeData)

,questionID,participant,Type1,Type2,Type3,timeInterval
0,7,KP1-no,Local,Control-flow,Ordering,76380.0
1,7,KP10-no,Local,Control-flow,Ordering,29575.0
2,7,KP11-no,Local,Control-flow,Ordering,85422.0
3,7,KP12-no,Local,Control-flow,Ordering,39223.0
4,7,KP13-no,Local,Control-flow,Ordering,144094.0
...,...,...,...,...,...,...
361,28,SP5-no,Global,Control-flow,Repetition,111184.0
362,28,SP6-no,Global,Control-flow,Repetition,77086.0
363,28,SP7-no,Global,Control-flow,Repetition,129895.0
364,28,SP8-no,Global,Control-flow,Repetition,60026.0


In [49]:
#Descriptives

In [50]:
responseTime(resTimeData).groupby(['Type1']).agg({"timeInterval":"mean"})

,timeInterval
Type1,
Global,117545.797814
Local,54107.737705


In [52]:
#V2
#Calculate mean per participant
resTime_new_part = responseTime(resTimeData).groupby(['participant','Type1','Type2'],as_index=False).agg({"timeInterval":"mean"})
#Calculate descriptives
resTime_new_part.groupby('Type1').agg({'timeInterval':'mean'})

,timeInterval
Type1,
Global,117592.971014
Local,54097.186594


In [70]:
#Inferentials

In [35]:
resTime_new_part = responseTime(resTimeData).groupby(['participant','Type1','Type2'],as_index=False).agg({"timeInterval":"mean"})

In [44]:
#Control-flow
resTime_part_cflow_Global = resTime_new_part.loc[(resTime_new_part["Type1"]=='Global')]
resTime_part_cflow_Local = resTime_new_part.loc[(resTime_new_part["Type1"]=='Local')]
resTime_part_cflow_merge = resTime_part_cflow_Global.merge(resTime_part_cflow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')


print(len(resTime_part_cflow_Global), len(resTime_part_cflow_Local))
print(len(resTime_part_cflow_merge))

stats.wilcoxon(resTime_part_cflow_merge['timeInterval_global'], resTime_part_cflow_merge['timeInterval_local'],alternative='greater')

46 46
46


WilcoxonResult(statistic=1071.0, pvalue=6.110667527536862e-13)

In [54]:
########################
#
# Perceived difficulty
#
#######################

In [55]:
#measure specific pre-processing

In [56]:
# select rows with questionEventType='questionOffset' and 'questionID' is in the questions considered for the analysis
percDifficultyData =  allData.loc[(allData['questionEventType']=='questionOffset') & (allData['questionID'].isin([str(question["id"]+2) for question in questions]))].copy(deep=True)
# connect the perceived difficulty to the parent QuestionID (remember each task has three questions, main question, verbal justification, perceived diff question)
percDifficultyData['questionID'] = percDifficultyData['questionID'].astype('int') - 2

#add question info
percDifficultyData = addQuestionInfo(percDifficultyData,questions)

#Select control-flow question type
percDifficultyData = percDifficultyData.loc[(percDifficultyData['Type2'] == 'Control-flow')] 

In [57]:
#Example of query: for each participant, we obtain one aggregated measure for local questions and one aggregate measure for global questions
perceivedDifficulty(percDifficultyData,['participant','Type1'])

,participant,Type1,difficultyScore
0,KP1-no,Global,1.00
1,KP1-no,Local,1.00
2,KP10-no,Global,3.00
3,KP10-no,Local,0.75
4,KP11-no,Global,2.75
...,...,...,...
87,SP7-no,Local,0.00
88,SP8-no,Global,1.50
89,SP8-no,Local,0.50
90,SP9-no,Global,2.50


In [58]:
#Descriptives

In [59]:
perceivedDifficulty(percDifficultyData,['Type1'])

,Type1,difficultyScore
0,Global,1.891304
1,Local,0.619565


In [60]:
#V2
#Calculate mean per participant
percDiff_part = perceivedDifficulty(percDifficultyData,['participant','Type1','Type2'])
#Calculate descriptives
percDiff_part.groupby('Type1').agg({'difficultyScore':'mean'})

,difficultyScore
Type1,
Global,1.891304
Local,0.619565


In [90]:
#Inferential

In [41]:
percDiff_part = perceivedDifficulty(percDifficultyData,['participant','Type1','Type2'])

In [45]:
percDiff_ControlFlow_Global = percDiff_part.loc[(percDiff_part["Type1"]=='Global')]
percDiff_ControlFlow_Local = percDiff_part.loc[(percDiff_part["Type1"]=='Local')]
percDiff_ControlFlow_merge = percDiff_ControlFlow_Global.merge(percDiff_ControlFlow_Local, on=['participant','Type2'], suffixes=('_global', '_local'), how='inner')

print(len(percDiff_ControlFlow_Global), len(percDiff_ControlFlow_Local))
print(len(percDiff_ControlFlow_merge))

stats.wilcoxon(percDiff_ControlFlow_merge['difficultyScore_global'],percDiff_ControlFlow_merge['difficultyScore_local'],alternative='greater')



46 46
46


C:\Users\la1949\Anaconda3\lib\site-packages\scipy\stats\_morestats.py:3255: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=946.0, pvalue=4.983743900200151e-09)